# Install needed library `aiohttp`
View video recording at:

https://moodle.imt-atlantique.fr/pluginfile.php/75912/mod_folder/content/0/Session04-install-anaconda-package.mkv?forcedownload=1

In summary:
* Run *Anaconda Navigator*
* Go to *Environments*
* Select your current enviroment (normally `root`)
* Select to show *Not installed* packages
* Search for the package `aiohttp`
* Tick the checkmark next to the package name
* Click *Apply*
* Wait for the installation
* Done

# Execute this whole notebook and go back to `Session04-01-requests-streaming-json.ipynb`

In [ ]:
import asyncio
from aiohttp import web
import subprocess
from pprint import pprint
import json
import random

class RandomDataGenerator:
    def __init__(self, seed, from_n, to_n):
        self.seed = seed
        self.from_n = from_n
        self.to_n = to_n
        self.rng = random.Random(self.seed)
        self.time_rng = random.Random(self.seed)

    def get_next(self):
        return self.rng.randint(self.from_n, self.to_n)

    def get_range(self):
        return self.from_n, self.to_n

    def set_range(self, from_n, to_n):
        self.from_n = from_n
        self.to_n = to_n
        
    def get_delay(self):
        return self.time_rng.randint(50, 400)/1000.0


rdg = RandomDataGenerator(1, 1, 10)


async def do_GET_random_number(request):
    N = int(request.query.get("count", '1000'))
    I = int(request.query.get("intervalms", '-1'))
    resp = web.StreamResponse(status=200, 
                              reason='OK', 
                              headers={'Content-Type': 'application/json'})
    
    await resp.prepare(request)
    n = 0
    while n < N:
        n += 1
        try:
            data = {
                'number': rdg.get_next()
            }
            await resp.write((json.dumps(data)+"\n").encode('utf-8'))
            delay = rdg.get_delay() if I<0 else I/1000.0
            await asyncio.sleep(delay)
        except Exception as e:
            print(repr(e))
            raise
    return resp


async def do_GET_random_range(request):
    resp = web.StreamResponse(status=200, 
                              reason='OK', 
                              headers={'Content-Type': 'application/json'})
    
    await resp.prepare(request)
    try:
        from_n, to_n = rdg.get_range()
        data = {
            'from_n': from_n,
            'to_n': to_n
        }
        await resp.write((json.dumps(data)+"\n").encode('utf-8'))
    except Exception as e:
        print(repr(e))
        raise
    return resp

async def do_POST_random_range(request):
    req_cont = await request.json()
    rdg.set_range(req_cont['from_n'], req_cont['to_n'])
    
    return await do_GET_random_range(request)

PORT = 8891

async def build_server(loop, address, port):
    app = web.Application()
    app.router.add_route('GET', "/random_number", do_GET_random_number)
    app.router.add_route('GET', "/random_range", do_GET_random_range)
    app.router.add_route('POST', "/random_range", do_POST_random_range)
    runner = web.AppRunner(app)
    await runner.setup()

    site = web.TCPSite(runner, address, port)
    await site.start()

loop = asyncio.get_event_loop()
server_future = asyncio.ensure_future(build_server(loop, '127.0.0.1', PORT))
print(f"Server ready on http://127.0.0.1:{PORT}")